In [1]:
import pandas as pd
import os
import requests as req
import json
import csv
import math
from datetime import datetime, timedelta
from datetime import date

import asyncio
import aiohttp

In [2]:
# OpenWeather API Key
API_KEY = 'c5d22ed423af74ba40fc97b49c023304'

In [3]:
# Read TRT data
data_frame = pd.read_csv('dimTRTstore.csv', header=0)

In [4]:
month_start = int(datetime(2022,11,1,1,0).timestamp())
month_end = int(datetime(2022,11,27,0,59,59).timestamp())

In [5]:
options = {}
options["weather_url"] = "https://history.openweathermap.org/data/2.5/history/city"
options["type"] = "hour"
options["start"] = str(month_start)
options["end"] = str(month_end)
options["units"] = "metric"
options["api_key"] = API_KEY

In [6]:
def hour_rounder(t):
    # Rounds down to the nearest hour
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour))

In [7]:
# Builds request url to access historical weather data from OpenWeather
# Params: 
# lat: latitude of desired location
# lon: longitude of desired location
# return: request url for weather

def request_weather_url(lat,lon):

    REQUEST_URL = options["weather_url"] + "?lat=" \
    + str(lat) \
    + "&lon=" + str(lon) \
    + "&type=" + options["type"] \
    + "&start=" + options["start"] \
    + "&units=" + options["units"] \
    + "&appid=" + options["api_key"]

    return REQUEST_URL

In [8]:
WEATHER_URL_LIST = []
for lat,lon in zip(data_frame["Latitude"], data_frame["Longitude"]):
    WEATHER_URL_LIST.append(request_weather_url(lat,lon))

In [9]:
async def fetch(session, url):
    async with session.get(url) as response:
        json_response = await response.json(content_type='application/json')
        json_response.update({'url': url})
        await asyncio.sleep(1)
        return json_response

In [10]:
async def main_weather(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in WEATHER_URL_LIST]
        return await asyncio.gather(*tasks)

In [11]:
result = await main_weather(WEATHER_URL_LIST)

In [12]:
df1 = pd.json_normalize(result)

In [13]:
data = df1[df1['list'].notna()]

In [17]:
data.to_parquet('testing.parquet', index=False)

In [16]:
data2 = pd.json_normalize(data.to_dict(orient='records'), record_path=['list','weather'], record_prefix='weather.')

In [17]:
data3 = pd.json_normalize(data.to_dict(orient='records'), record_path=['list'], meta='url')

In [14]:
data = df1.to_numpy().tolist()

In [15]:
dn = pd.json_normalize(data['list'])

In [14]:
dw = pd.json_normalize(result, 'list', meta=['url'], errors='ignore')
dn = pd.json_normalize(result['list'], record_path=['weather'], record_prefix='weather.' )
dw = dw.drop('weather', axis=1)
dw['dt'] = dw['dt'].apply(lambda t: hour_rounder(datetime.utcfromtimestamp(t)).strftime('%Y-%m-%d %H:%M:%S'))
df_update = pd.concat([dw,dn], axis=1)
df_update['lat'] = df_update['url'].apply(lambda t: t.split('lat=')[1].split('&lon')[0])
df_update['lon'] = df_update['url'].apply(lambda t: t.split('lon=')[1].split('&type')[0])
df_update = df_update.drop('url', axis=1)

KeyError: "Key 'list' not found. If specifying a record_path, all elements of data should have the path."